In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from requests.exceptions import HTTPError
import sys
import math

In [3]:
# url = "http://www.nepalstock.com/"
# request = requests.get(url)
# soup = BeautifulSoup(request.text, 'lxml')
# total_transactions = soup.find(text="Total Transactions")
# total_transactions = total_transactions.parent.parent.next_sibling.next_sibling.text
# total_transactions = total_transactions.replace(",", "")
# total_iteration = math.ceil(int(total_transactions)/500)
# print(total_iteration)
# print(request.text)


In [4]:

# table_list = list()
# # subgroup = "floorsheet"
# # # url = "http://iloveyou:8000"
# # r_list = list()
# # mistake_count = 0
# # for index in range(1,total_iteration+1):
# #     url = f"http://www.nepalstock.com/main/floorsheet/index/{index}/?contract-no=&stock-symbol=&buyer=&seller=&_limit=500/"
# #     while True:
# #         try:
# #             r = requests.get(url)
# #             if r.status_code == 200 :  
# #                 break
# #         except requests.exceptions.RequestException as e:  # This is the correct syntax
# #             mistake_count += 1
# #             if mistake_count > 3:
# #                 print(e)
# #                 sys.exit()
# #                 continue
        
# #     r_list.append(r)
    
    


In [4]:
# r.url
# for x in r_list:
#     print(x.url)



In [1]:

# for r in r_list:
#     soup = BeautifulSoup(r.text, "lxml")
#     table = soup.find("table", class_="table my-table").contents
#     table = list(filter(lambda a : a!='\n', table))
#     # row = [x.text for x in table[2].find_all("td")]
#     # print(row)
    
#     for row in table[2:-3]:
#         datas = row.find_all("td")
#         row = [x.text for x in datas]
#         row[5:] = [float(x) for x in row[5:]]
#         table_list.append(row)

#     # data_json = json.dumps(table_list)
#     # print(data_json)
# columns = ['S.N.', 'Contract No','Stock Symbol', 'Buyer Broker',
#            'Seller Broker', 'Quantity', 'Rate', 'Amount']
# df = pd.DataFrame(table_list, columns=columns)
# df.to_csv("floorsheet.csv")



NameError: name 'r_list' is not defined

In [2]:


df = pd.read_csv('./datas/floorsheet_2021-06-20_Sunday.csv', low_memory=False)
df.head(50)
df.drop_duplicates()

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
0,0,1,NLICL,45,43,79,1235.0,97565.0
1,1,1,NLICL,45,53,15,1235.0,18525.0
2,2,1,BOKL,8,28,290,377.0,109330.0
3,3,1,BOKL,8,45,10,377.0,3770.0
4,4,1,NBL,10,33,90,447.0,40230.0
...,...,...,...,...,...,...,...,...
128276,128276,257,CBBL,48,45,50,1750.0,87500.0
128277,128277,257,CBBL,14,45,20,1750.0,35000.0
128278,128278,257,CBBL,57,45,20,1750.0,35000.0
128279,128279,257,SCB,58,49,200,604.0,120800.0


In [32]:

# df.tail(10).style.hide_index()

df[(df["Amount"] > 10000000) ].sort_values("Amount", ascending =False)

# df[(df["Amount"] > 1000000)].sort_values("Amount", ascending =False)

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
24377,24377,49,NRIC,58,54,8452,1890.0,15974280.0
17966,17966,36,RHPC,3,58,18827,600.0,11296200.0
11827,11827,24,GMFIL,58,42,19399,520.0,10087480.0


In [13]:
# c1 = df["Stock Symbol"] 
df.shape



(68000, 8)

In [12]:
c2 = df["Buyer"]=="39"
c3 = df["Seller"]== "58"
# order by broker
c1 = df["Amount"] > 10000
c4 = df["Symbol"] == "GUFL"
df[c1].sort_values("Amount", ascending=False).head(20)
# df[c2].groupby([ "Buyer Broker", "Stock Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
99708,99708,200,IGI,42,5,28930,1310.00,37898300.0
99071,99071,199,GBIME,42,42,50000,448.00,22400000.0
105461,105461,211,MBL,44,33,43603,396.00,17266788.0
96960,96960,194,CBL,49,50,58875,292.00,17191500.0
120514,120514,242,GBIMEP,5,42,85000,153.00,13005000.0
15920,15920,32,NTC,58,36,10000,1275.00,12750000.0
49353,49353,99,HDL,35,58,2000,5200.00,10400000.0
123814,123814,248,GIMES1,43,43,500000,19.65,9825000.0
53398,53398,107,NRIC,44,4,5000,1880.00,9400000.0
127814,127814,256,GIMES1,34,34,444322,19.70,8753143.4


In [8]:
c3 = df["Buyer"]== "58"
df[c3].groupby([ "Buyer", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Buyer  Symbol
58     API       13186086.0
       CHDC       6810380.0
       ALICL      5805602.0
       AKPL       3155089.0
       BOKL       2944325.0
       AHPC       2900420.0
       CCBL       2595993.0
       BPCL       2518871.0
       CBL        2327212.0
       BFC        2137301.0
       CBBL       1693277.0
       AIL        1174420.0
       CFCL       1158960.0
       CGH        1082910.0
       CHCL        748522.0
       BARUN       617590.0
       ADBL        599542.0
       AKJCL       451197.0
       ACLBSL      129030.0
       ALBSL        57640.0
Name: Amount, dtype: float64

In [9]:
c = df["Buyer"] == "58"
df[c].groupby([ "Seller", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Seller  Symbol
1       CIT       7180460.0
        CBBL       500460.0
        GBBL       326450.0
        API        143020.0
        CFCL       112780.0
        CHDC        72220.0
        GIC         67740.0
        AIL         43950.0
        BPCL        29860.0
        DDBL        26025.0
        BOKL        24507.0
        BFC         23128.0
        GBIME       19756.0
        CBL         16060.0
        ADBL        12225.0
        DHPL        11160.0
        CZBIL        7880.0
        GFCL         6830.0
        AKPL         6084.0
        CMF2         4038.0
Name: Amount, dtype: float64

In [5]:
c = df["Symbol"] == "GUFL"
df[c].groupby([ "Symbol", "Buyer"])["Amount"].sum().head(20).sort_values(ascending=False)

Symbol  Buyer
GUFL    20       36769427.0
        3        11496746.0
        44        6217845.0
        28        4615009.0
        35        3484102.0
        36        1303450.0
        14        1059092.0
        17         985070.0
        38         873290.0
        34         727700.0
        19         685300.0
        37         566500.0
        43         452361.0
        33         384784.0
        16         287500.0
        21         246900.0
        22         237150.0
        42         164145.0
        41          67617.0
        11          45240.0
Name: Amount, dtype: float64

In [5]:
c2 = df["Seller"]== "58"
df[c2].groupby([ "Seller", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Seller  Symbol
58      AHPC      20536977.0
        BOKL       7362772.0
        CHCL       3725780.0
        API        2764861.0
        BFC        1794517.0
        CBL        1635152.0
        BPCL       1583385.0
        ALICL      1503346.0
        BARUN       881840.0
        ADBL        852088.0
        CCBL        720012.0
        CFCL        702083.0
        AKPL        650620.0
        CBBL        457440.0
        AIL         413696.0
        CGH         369650.0
        AKJCL       233735.0
        BBC          63760.0
        ALBSL        21600.0
        ACLBSL       16050.0
Name: Amount, dtype: float64

In [13]:
#top turnover
df[df["Symbol"] == "GUFL"].groupby(["Symbol"])["Amount"].sum().sort_values(ascending = False).head(20)
# df["Amount"].sum()

Symbol
GUFL    90542388.0
Name: Amount, dtype: float64